In [1]:
# Importação de bibliotecas

import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [2]:
# Carregamento dos dados do arquivo CSV

df = pd.read_csv("../data/raw/titanic.csv")
display(df.head())

# Carregamento do dicionário de dados

df_dict = pd.read_csv("../data/external/dictionary.csv")
display(df_dict)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


,variavel,descrição,tipo,subtipo
0,survived,Indica se o passageiro sobreviveu (sim = 1 ou ...,qualitativa,nominal
1,pclass,"Classe do ticket do passageiro (1 = 1ª, 2 = 2ª...",qualitativa,ordinal
2,sex,Sexo do passageiro,qualitativa,nominal
3,age,Idade do passageiro,quantitativa,contínua
4,sibsp,Número de irmãos/cônjuges a bordo,quantitativa,discreta
5,parch,Número de pais/filhos a bordo,quantitativa,discreta
6,fare,Tarifa paga pelo passageiro,quantitativa,contínua
7,embarked,Porto de embarque (C = Cherbourg; Q = Queensto...,qualitativa,nominal
8,class,"Classe do ticket (Primeira, Segunda, Terceira)",qualitativa,ordinal
9,who,"Descrição da pessoa (homem, mulher, criança)",qualitativa,nominal


In [3]:
# Definição da variável algo

target_variable = 'survived'

# Seleciona variáveis quantitativas, excluindo a variável alvo

quantitative_variables = (
    df_dict
    .query("tipo == 'quantitativa' and variavel != @target_variable")
    .variavel
    .to_list()
)

# Seleciona variáveis qualitativas, excluindo a variável alvo

qualitative_variables = (
    df_dict
    .query("tipo == 'qualitativa' and variavel != @target_variable")
    .variavel
    .to_list()
)

In [4]:
# Removendo a coluna da variável alvo do DataFrame original
X = df.drop(columns=[target_variable])

# Criar o DataFrame contendo apenas a coluna da variável alvo do DataFrame original
y = df[[target_variable]]

In [5]:
# tratamento de dados discrepantes

# Pipeline para pré-processamento de variáveis quantitativas

quantitative_preprocess = Pipeline([
    ('missing', SimpleImputer(strategy='median')), 
    ('normalization', StandardScaler()), 
])

# Pipeline para pré-processamento de variáveis qualitativas

qualitative_preprocess = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')), 
    ('encoding', OneHotEncoder()), 
    # ('normalization', StandardScaler()),
])

# Combinação dos pipelines para pré-processamento de todas as variáveis

preprocess = ColumnTransformer([
    ('quantitative', quantitative_preprocess, quantitative_variables),
    ('qualitative', qualitative_preprocess, qualitative_variables)
])